In [113]:
import pandas as pd
import numpy as np
from pandas.core.interchange.dataframe_protocol import DataFrame
from pygments.lexer import bygroups

import volpy_func_lib as vp
import load_clean_lib
import importlib
from datetime import datetime
import matplotlib.pyplot as plt

import table_lib

from datetime import datetime

from volpy_func_lib import load_forward_price

In [114]:
# 3 cleaning bud:

# mid price for option > x  (x=~0.1)
# volumen > x
# bid ask spread / mid price < x

In [115]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)

<module 'table_lib' from 'D:\\Professionelt\\Code\\volpy\\table_lib.py'>

# Lav Data

## Load, Clean and Prepare data

In [116]:
# Load data
profile = "Mads"
dataset = "i4s4"
od, FW, ZCY_curves, returns_and_prices = load_clean_lib.load_od_FW_ZCY(profile, dataset)

# first_day = pd.to_datetime("1996-01-04")
# last_day = pd.to_datetime("1997-01-01")
first_day = pd.to_datetime("1996-01-04")
# first_day = pd.to_datetime("2002-01-04")
last_day = pd.to_datetime("2003-02-28")
# last_day = pd.to_datetime("1996-05-01")

# Clean data
od = load_clean_lib.clean_od(od, first_day = first_day, last_day = last_day)

## Realized vol calculation

In [117]:
real_vol = vp.calc_realized_var(returns_and_prices, first_day, last_day)

### Add FW to options

In [118]:
od = vp.add_FW_to_od(od, FW)

### remove if ITM

In [119]:
od = od.loc[
    ((od["F"] < od["K"]) & (od["cp_flag"] == "C")) | ((od["F"] > od["K"]) & (od["cp_flag"] == "P"))
]

In [120]:
# od = od[od["volume"] > 0]

### Add r to options

In [121]:
od = vp.add_r_to_od_parallel(od, ZCY_curves)

## Add low/high and create summary (Filters dataset for criteria such as min 3 strikes ... min 8 days...)

In [122]:
od, summary_dly_df = vp.od_filter_and_summary_creater(od)

### only keep the lowest ("low") and the second lowest ("high") TTMs

In [123]:
od_rdy = od[(od["low"] == True) | (od["high"] == True)]

## Calculate swap rate

In [124]:
summary_dly_df = vp.fill_swap_rates(summary_dly_df, od_rdy, n_points=200)

Processing Groups: 100%|██████████| 26686/26686 [00:15<00:00, 1737.16it/s]


## Save/Load CSV

In [125]:
# # Save
# date_str = datetime.now().strftime("%Y-%m_%d_%H-%M")
# summary_dly_df.to_csv(f"data/summary_dly_df {date_str}.csv")
# od_rdy.to_csv(f"data/od_rdy {date_str}.csv")

# # Load
# summary_dly_df = pd.read_csv(f"data/summary_dly_df {date_str}.csv")
# od_rdy = pd.read_csv(f"data/od_rdy {date_str}.csv")

## Keep only active days, and take average for table (just overview)

In [126]:
# Select only the numeric columns in the DataFrame.
numeric_cols = summary_dly_df.select_dtypes(include=[np.number]).columns

# Filter rows where "low SW" is not NaN, group by "Ticker", and calculate the mean for numeric columns.
filtered_df = summary_dly_df[summary_dly_df["Active"] == True]
avg_by_ticker = filtered_df.groupby("Ticker")[numeric_cols].mean()
avg_by_ticker

,#days,low days,high days,low #K,high #K,#K,low SW,high SW,SW
Ticker,,,,,,,,,
AMZN,4.033307,23.758128,62.854877,11.880254,15.962728,13.921491,1.006674,0.980833,1.018289
DJX,5.850295,23.362832,53.561947,17.850295,18.008850,17.929572,0.065117,0.067193,0.065274
GE,4.591778,23.635785,61.432542,5.697742,7.723798,6.710770,0.127272,0.121146,0.124286
IBM,4.106229,23.354839,62.072859,9.722469,12.868743,11.295606,0.163329,0.151445,0.159358
MSFT,4.105673,23.356507,62.078420,8.860400,12.284205,10.572303,0.186426,0.179673,0.184420
NDX,4.696835,23.337590,53.949473,37.039423,36.615769,36.827596,0.176892,0.173006,0.175524
OEX,4.479178,23.337590,53.588007,33.555802,30.267074,31.911438,0.060220,0.061311,0.060750
SPX,5.829539,23.337590,53.603554,33.886174,29.960022,31.923098,0.058093,0.058616,0.058229


# Replications of Carr & Wu 2009 Tables

### Merge Realized Vol onto Variance Premium

In [144]:
# Reset index so that 'Date' and 'Ticker' become columns
summary_dly_df_reset = summary_dly_df.reset_index()

# Merge by specifying left_on and right_on keys
merged_df = summary_dly_df_reset.merge(
    real_vol[['date', 'ticker', 'RV', 'open']],
    left_on=['Date', 'Ticker'],
    right_on=['date', 'ticker'],
    how='left'
)

# Optionally, set the index back to ['Date', 'Ticker']
summary_dly_df = merged_df.set_index(['Date', 'Ticker'])
summary_dly_df = summary_dly_df.reset_index()

## Table 1 (Aktive dage)

In [168]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)

<module 'table_lib' from 'D:\\Professionelt\\Code\\volpy\\table_lib.py'>

In [170]:
importlib.reload(table_lib)
table_lib.CarrWu2009_table_1(summary_dly_df, print_latex = True)

\begin{tabular}{lllrr}
\toprule
Ticker & Starting date & Ending date & N & NK \\
\midrule
SPX & 04-Jan-1996 & 28-Feb-2003 & 1801 & 31.92 \\
OEX & 04-Jan-1996 & 28-Feb-2003 & 1801 & 31.91 \\
DJX & 06-Oct-1997 & 28-Feb-2003 & 1356 & 17.93 \\
NDX & 04-Jan-1996 & 28-Feb-2003 & 1801 & 36.83 \\
MSFT & 04-Jan-1996 & 28-Feb-2003 & 1798 & 10.57 \\
IBM & 04-Jan-1996 & 28-Feb-2003 & 1798 & 11.30 \\
GE & 08-Jan-1996 & 28-Feb-2003 & 1727 & 6.71 \\
AMZN & 20-Nov-1997 & 28-Feb-2003 & 1261 & 13.92 \\
\bottomrule
\end{tabular}



,Ticker,Starting_date,Ending_date,N,NK
7,SPX,04-Jan-1996,28-Feb-2003,1801,31.923098
6,OEX,04-Jan-1996,28-Feb-2003,1801,31.911438
1,DJX,06-Oct-1997,28-Feb-2003,1356,17.929572
5,NDX,04-Jan-1996,28-Feb-2003,1801,36.827596
4,MSFT,04-Jan-1996,28-Feb-2003,1798,10.572303
3,IBM,04-Jan-1996,28-Feb-2003,1798,11.295606
2,GE,08-Jan-1996,28-Feb-2003,1727,6.710770
0,AMZN,20-Nov-1997,28-Feb-2003,1261,13.921491


## Table 2 (SW og RV)

In [173]:
table_lib.CarrWu2009_table_2(summary_dly_df, print_latex = True) # , print_latex = True

\begin{tabular}{lrrrrrrrrrr}
\toprule
 & \multicolumn{5}{r}{Panel A: Realized variance, RV×100} & \multicolumn{5}{r}{Panel B: Variance swap rate, SW×100} \\
Ticker & Mean & Std. dev. & Auto & Skew & Kurt & Mean & Std. dev. & Auto & Skew & Kurt \\
\midrule
SPX & 4.10 & 3.36 & 0.80 & 2.11 & 5.10 & 5.82 & 3.18 & 0.79 & 1.81 & 4.15 \\
OEX & 4.57 & 3.74 & 0.84 & 2.03 & 4.65 & 6.07 & 3.21 & 0.83 & 1.62 & 2.90 \\
DJX & 4.44 & 3.64 & 0.73 & 2.10 & 4.23 & 6.53 & 3.43 & 0.61 & 1.46 & 8.21 \\
NDX & 16.46 & 15.68 & 0.88 & 2.51 & 8.17 & 17.55 & 10.82 & 0.93 & 0.99 & 0.37 \\
MSFT & 16.70 & 13.28 & 0.68 & 2.34 & 6.25 & 18.44 & 9.83 & 0.79 & 2.16 & 6.92 \\
IBM & 15.07 & 11.13 & 0.72 & 1.73 & 3.42 & 15.94 & 7.83 & 0.77 & 1.88 & 4.40 \\
GE & 10.98 & 8.66 & 0.83 & 2.15 & 5.75 & 12.43 & 6.94 & 0.87 & 1.28 & 1.74 \\
AMZN & 92.27 & 60.32 & 0.78 & 1.01 & 0.76 & 101.83 & 55.34 & 0.87 & 1.58 & 2.94 \\
\bottomrule
\end{tabular}



,Ticker,Mean_RV,Std_RV,Auto_RV,Skew_RV,Kurt_RV,Mean_SW,Std_SW,Auto_SW,Skew_SW,Kurt_SW
7,SPX,4.096837,3.364648,0.801873,2.105232,5.104466,5.822859,3.181829,0.794188,1.813065,4.149287
6,OEX,4.567990,3.743152,0.836468,2.028253,4.654040,6.074986,3.210560,0.827144,1.617449,2.901213
1,DJX,4.444232,3.641170,0.731719,2.104698,4.232699,6.527394,3.430674,0.614444,1.455293,8.209319
5,NDX,16.455004,15.682873,0.882505,2.506586,8.170743,17.552373,10.816016,0.928520,0.987388,0.371159
4,MSFT,16.703479,13.279006,0.683582,2.339520,6.248022,18.442002,9.826454,0.789646,2.161156,6.921030
3,IBM,15.070294,11.126459,0.720023,1.734372,3.417668,15.935827,7.833802,0.767843,1.877459,4.400490
2,GE,10.984044,8.655460,0.832538,2.150305,5.747247,12.428584,6.936464,0.873766,1.278210,1.743881
0,AMZN,92.269480,60.324878,0.782890,1.006889,0.764826,101.828868,55.343045,0.871124,1.584801,2.940773


## Table 3 (SW - RV og SW/RV)

In [172]:
importlib.reload(table_lib)
table_lib.CarrWu2009_table_3(summary_dly_df, print_latex = True)

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & \multicolumn{6}{r}{Panel A: (RV - SW) x 100} & \multicolumn{6}{r}{Panel B: ln(RV / SW)} \\
Ticker & Mean & Std. dev. & Auto & Skew & Kurt & t & Mean & Std. dev. & Auto & Skew & Kurt & t \\
\midrule
SPX & -1.73 & 3.04 & 0.72 & 0.22 & 4.03 & -7.00 & -0.49 & 0.54 & 0.74 & 0.31 & -0.00 & -10.81 \\
OEX & -1.51 & 3.26 & 0.77 & 0.74 & 3.19 & -5.63 & -0.44 & 0.55 & 0.79 & 0.38 & -0.08 & -9.46 \\
DJX & -2.09 & 3.79 & 0.64 & 0.65 & 7.84 & -6.28 & -0.53 & 0.57 & 0.73 & 0.71 & 0.53 & NaN \\
NDX & -1.10 & 10.09 & 0.70 & 2.47 & 12.24 & -1.43 & -0.21 & 0.44 & 0.73 & 0.48 & 0.53 & -6.29 \\
MSFT & -1.74 & 9.99 & 0.51 & 1.83 & 7.12 & -2.34 & -0.22 & 0.48 & 0.60 & 0.06 & 0.57 & -6.15 \\
IBM & -0.87 & 9.12 & 0.66 & 1.15 & 2.52 & -1.27 & -0.19 & 0.57 & 0.67 & -0.02 & -0.10 & -4.39 \\
GE & -1.31 & 6.59 & 0.71 & 2.15 & 12.27 & -2.80 & -0.21 & 0.45 & 0.68 & 0.51 & 0.85 & -6.66 \\
AMZN & -9.18 & 53.71 & 0.70 & 0.44 & 1.74 & -1.79 & -0.20 & 0.56 & 0.72 & 0.05 & -0.15 &

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Ticker,Mean_diff,Std_diff,Auto_diff,Skew_diff,Kurt_diff,t_diff,Mean_ln,Std_ln,Auto_ln,Skew_ln,Kurt_ln,t_ln
7,SPX,-1.726022,3.038638,0.719158,0.224609,4.032837,-7.001344,-0.494814,0.544391,0.737009,0.311612,-0.001852,-10.808633
6,OEX,-1.506996,3.259668,0.773874,0.742657,3.187962,-5.627873,-0.436230,0.546871,0.792233,0.377617,-0.076043,-9.457162
1,DJX,-2.085289,3.786799,0.635105,0.648692,7.835276,-6.279278,-0.527805,0.567047,0.726221,0.705848,0.526587,NaN
5,NDX,-1.097369,10.085903,0.700185,2.468369,12.244608,-1.431133,-0.214685,0.444255,0.730220,0.475855,0.525207,-6.288701
4,MSFT,-1.744507,9.994447,0.508115,1.830105,7.124462,-2.338059,-0.221294,0.481739,0.604980,0.061394,0.572602,-6.148073
3,IBM,-0.869573,9.115209,0.656524,1.153213,2.519962,-1.266673,-0.194394,0.567729,0.673305,-0.020205,-0.095763,-4.386986
2,GE,-1.309596,6.593051,0.713798,2.147570,12.268656,-2.802234,-0.207820,0.448082,0.684016,0.507542,0.845170,-6.660722
0,AMZN,-9.182990,53.709431,0.695076,0.435074,1.744367,-1.793037,-0.197041,0.557300,0.723107,0.052333,-0.145504,-3.446732


## Table 4 (CAPM alpha regression)

## Table 5 (Fama-French alpha regression)

## Table 6 (Constant SW premium test)

## Table 10 (2 subsamples SW/RV ect.)